# Accurate and Efficient Protein-Peptide Docking with DiffPepDock

<img src="https://github.com/YuzheWangPKU/DiffPepBuilder/blob/main/examples/figures/dpd_model.jpg?raw=true">

This notebook demonstrates how to use the DiffPepDock tool to dock batches of peptide sequences to a specific target protein. We provide an example of the redocking task of the substrate-binding protein YejA in complex with its native peptide fragment (PDB ID: [7Z6F](https://www.rcsb.org/structure/7Z6F)) to demonstrate the procedures of batch docking.

## Setup

In [1]:
#@title ### Download model assets
import os

diffpep_folder = "DiffPepBuilder"
checkpoint_file = os.path.join(diffpep_folder, "experiments", "checkpoints", "diffpepdock_v1.pth")

if not (os.path.isdir(diffpep_folder) and os.path.isfile(checkpoint_file)):
  print("Installing DiffPepDock...")

  if not os.path.isdir(diffpep_folder):
    os.system("git clone https://github.com/YuzheWangPKU/DiffPepBuilder.git")

  os.chdir(diffpep_folder)

  if not os.path.isfile(checkpoint_file):
    print("Downloading model weights...")
    os.system("wget https://zenodo.org/records/15398020/files/diffpepdock_v1.pth")
    os.makedirs("experiments/checkpoints/", exist_ok=True)
    os.system("mv diffpepdock_v1.pth experiments/checkpoints/")

  os.chdir("..")
  print("DiffPepDock is installed and ready.")

else:
  print("DiffPepDock is already installed and ready.")

Installing DiffPepDock...
DiffPepDock is installed and ready.


In [2]:
#@title ### Install dependencies
os.system("pip install wget wandb fair-esm biotite pyrootutils easydict biopython tqdm ml-collections mdtraj GPUtil dm-tree tmtools py3Dmol")
os.system("wget https://west.rosettacommons.org/pyrosetta/release/release/PyRosetta4.MinSizeRel.python311.linux.wheel/pyrosetta-2024.39+release.59628fb-cp311-cp311-linux_x86_64.whl")
os.system("pip install pyrosetta-2024.39+release.59628fb-cp311-cp311-linux_x86_64.whl")
os.system("rm pyrosetta-2024.39+release.59628fb-cp311-cp311-linux_x86_64.whl")

pdbfixer_folder = "pdbfixer"
if not os.path.isdir(pdbfixer_folder):
  print("Installing pdbfixer...")
  os.system("git clone https://github.com/openmm/pdbfixer.git")
  os.chdir(pdbfixer_folder)
  os.system("python setup.py install")
  os.chdir("..")
  print("pdbfixer is installed.")
else:
  print("pdbfixer is already cloned.")

os.system("pip install hydra-core hydra-joblib-launcher")

Installing pdbfixer...
pdbfixer is installed.


0

## Inference

In [ ]:
#@title ### Specify receptor information
from google.colab import files
import json

os.makedirs("test_case", exist_ok=True)
receptor_type = "default (7Z6F)" #@param ["default (7Z6F)", "uploaded"]

if receptor_type == "uploaded":
  uploaded_pdb = files.upload()
  file_name = next(iter(uploaded_pdb))
  if not file_name.endswith(".pdb"):
    raise ValueError("Please upload in PDB format.")
  os.system(f"cp {file_name} test_case/")
  os.system(f"cp test_case/{file_name} DiffPepBuilder/examples/docking_data/")
else:
  file_name = "7Z6F.pdb"
  os.system(f"cp DiffPepBuilder/examples/docking_data/7Z6F.pdb test_case/")
#@markdown - **Note**: please remove non-protein components from the PDB file and ensure that the CA atoms are present.

lig_chain = "A" #@param {type:"string"}
#@markdown  - Chain ID of the **reference** ligand. Please set to `None` if no reference ligand is included in the PDB file.
#@markdown  The model will prioritize reference ligand information over the binding motif if both are given.
motif = None #@param {type:"string"}

key = os.path.splitext(file_name)[0]
data = {}
if lig_chain and lig_chain != "None":
  data['lig_chain'] = lig_chain
if motif and motif != "None":
  data['motif'] = motif.replace(",", "-")

json_file_write_path = "test_case/docking_cases.json"
final_data = {key: data}
with open(json_file_write_path, 'w') as file:
  json.dump(final_data, file, indent=4)

In [4]:
#@title ### Specify peptide sequences
import ipywidgets
from IPython.display import display, clear_output

mode_dropdown = ipywidgets.Dropdown(
    options=['single', 'batch'],
    value='single',
    description='Docking Mode:',
    style={'description_width': 'initial'}
)

# Widgets for SINGLE mode
peptide_seq = ipywidgets.Text(
    value='VLGEPRYAFNFN',
    description='Peptide Sequence:',
    style={'description_width': 'initial'}
)
peptide_id = ipywidgets.Text(
    value='nat',
    description='Peptide ID:',
    style={'description_width': 'initial'}
)
save_button = ipywidgets.Button(
    description='Save Sequence',
    button_style='success'
)

# Widgets for BATCH mode
upload_widget = ipywidgets.FileUpload(
    accept='.fasta',
    multiple=False,
    description='Upload FASTA'
)
process_button = ipywidgets.Button(
    description='Process Upload',
    button_style='info'
)

output = ipywidgets.Output()

def on_mode_change(change):
    with output:
        clear_output(wait=True)
        if change['new'] == 'single':
            display(ipywidgets.VBox([peptide_seq, peptide_id, save_button]))
        else:
            display(ipywidgets.VBox([upload_widget, process_button]))

def save_sequence(btn):
    seq = peptide_seq.value.strip()
    pid = peptide_id.value.strip()
    if not seq or not pid:
        with output:
            print("Please provide both Sequence and ID.")
        return
    path = os.path.join("test_case", "peptide_seq.fasta")
    with open(path, 'w') as f:
        f.write(f">{pid}\n{seq}\n")
    with output:
        print(f"Saved single-sequence file to {path}")

def process_upload(btn):
    if not upload_widget.value:
        with output:
            clear_output(wait=True)
            print("No FASTA file uploaded.")
        return
    with output:
        clear_output(wait=True)
        for fname, fileinfo in upload_widget.value.items():
            dest = os.path.join("test_case", "peptide_seq.fasta")
            with open(dest, 'wb') as f:
                f.write(fileinfo['content'])
            print(f"Saved batch file to {dest}")

mode_dropdown.observe(on_mode_change, names='value')
save_button.on_click(save_sequence)
process_button.on_click(process_upload)

display(mode_dropdown, output)
on_mode_change({'new': mode_dropdown.value})

Dropdown(description='Docking Mode:', options=('single', 'batch'), style=DescriptionStyle(description_width='i…

Output()

In [5]:
#@title ### Preprocess receptor and peptide sequence data
!python DiffPepBuilder/experiments/process_batch_dock.py \
  --pdb_dir test_case \
  --write_dir test_case \
  --receptor_info_path test_case/docking_cases.json \
  --peptide_seq_path test_case/peptide_seq.fasta

Files will be written to test_case
Finished test_case/7Z6F.pdb in 0.23s
Finished processing 1/1 files. Start ESM embedding...
Model file /content/DiffPepBuilder/experiments/checkpoints/esm2_t33_650M_UR50D.pt not found. Downloading...
Model file /content/DiffPepBuilder/experiments/checkpoints/esm2_t33_650M_UR50D-contact-regression.pt not found. Downloading...
Read sequence data with 2 sequences
Processing protein sequence batches:   0% 0/1 [00:00<?, ?it/s]Processing 1 of 1 batches (2 sequences)
Processing protein sequence batches: 100% 1/1 [00:01<00:00,  1.42s/it]
1it [00:00, 102.51it/s]


In [ ]:
#@title ### Customize docking settings
import yaml

#@markdown #### Sampling params
denoising_steps = "200" #@param [100, 200, 500]
noise_scale = "1" #@param [0.5, 1, 1.5, 2, 2.5]
samples_per_sequence = 4 #@param {type:"integer"}

#@markdown #### Postprocessing params
amber_relax = True #@param {type:"boolean"}
#@markdown  - Amber relaxation runs slowly on Colab. You may skip this step for faster inference with minimal quality loss.
rosetta_relax = True #@param {type:"boolean"}

yaml_file_path = "DiffPepBuilder/config/docking.yaml"
with open(yaml_file_path, 'r') as file:
  yaml_data = yaml.safe_load(file)

yaml_data['data']['num_t'] = int(denoising_steps)
yaml_data['experiment']['noise_scale'] = float(noise_scale)
yaml_data['experiment']['eval_batch_size'] = int(samples_per_sequence)
yaml_data['data']['num_repeat_per_eval_sample'] = int(samples_per_sequence)

yaml_data['postprocess']['amber_relax'] = amber_relax
yaml_data['postprocess']['rosetta_relax'] = rosetta_relax

with open(yaml_file_path, 'w') as file:
  yaml.dump(yaml_data, file, default_flow_style=False)


In [7]:
#@title ### Run docking
#@markdown - Postprocessing will be performed automatically after docking.
os.environ['BASE_PATH'] = "/content/DiffPepBuilder"

!torchrun --nproc-per-node=1 DiffPepBuilder/experiments/run_docking.py \
  data.val_csv_path=test_case/metadata_test.csv \
  experiment.use_ddp=False \
  experiment.num_gpus=1 \
  experiment.num_loader_workers=1

[2025-06-05 18:38:52,053][experiments.train][INFO] - Loading checkpoint from /content/DiffPepBuilder/experiments/checkpoints/diffpepdock_v1.pth
[2025-06-05 18:38:57,486][data.so3_diffuser][INFO] - Computing IGSO3. Saving in /content/DiffPepBuilder/runs/.cache/eps_1000_omega_1000_min_sigma_0_1_max_sigma_1_5_schedule_logarithmic
[2025-06-05 18:40:12,126][experiments.train][INFO] - Number of model parameters: 103.66 M
[2025-06-05 18:40:17,202][experiments.train][INFO] - Evaluation mode only, no checkpoint being saved.
[2025-06-05 18:40:17,205][experiments.train][INFO] - Evaluation saved to: /content/DiffPepBuilder/runs/docking/05D_06M_2025Y_18h_40m
[2025-06-05 18:40:17,336][experiments.train][INFO] - Using device: cuda:0
[2025-06-05 18:40:17,342][data.pdb_data_loader][INFO] - Validation: 1 examples
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:508: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. We recommen

In [8]:
#@title ### Download results
#@markdown - The docking results will be saved in the `DiffPepBuilder/runs` folder.

!tar --directory=/content/DiffPepBuilder/runs -czf /content/docking_results.tar.gz docking
files.download("/content/docking_results.tar.gz")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>